In [5]:
from dotenv import load_dotenv

import os

os.chdir(r'C:\Users\Josh\Documents\Git\crewai')

load_dotenv()

import os
from crewai import Agent, Task, Crew, Process
from langchain.tools import BaseTool
from langchain.agents import load_tools, initialize_agent, Tool
from langchain_community.document_loaders import PyPDFLoader
from langchain import VectorDBQA, OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

# Loading Human Tools
human_tools = load_tools(["human"])

# Create RAG tool

loader = PyPDFLoader("policy3.pdf")
texts = loader.load_and_split()
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)
llm=OpenAI(model='gpt-4-0125-preview', temperature=0)
vec_DBQA = RetrievalQA.from_chain_type(llm=llm, retriever=docsearch.as_retriever())

tools = [
    Tool(
        name="Company_expenditure_policy",
        func=vec_DBQA.run,
        description="useful for when you need to analyse our expenditure governance policies"
    ),
]

# Define your agents with roles and goals

whitehat  = Agent(
  role='WhiteHat Finance/Risk/Insurance Analyst',
  goal='Analyze the terms and conditions of insurance policies using the information provided and find loopholes, exploits, and other means that bad actors could use to bypass our policies',
  backstory="""You are an ethical white hat security analyst with
  many years of experience analyzing policies and frameworks and finding ways that bad actors (e.g. fraudsters) may exploit them.
  """,
  verbose=True,
  allow_delegation=False,

)

risk_analyst  = Agent(
  role='Financial Risk Analyst',
  goal='Help companies patch up loopholes and other issues with their policy frameworks to prevent fraud',
  backstory="""You are a Financial Risk Analyst with years of experience.
  Your primary responsibility is to develop ways to revise and strengthen a companies risk posture.""",
  verbose=True,
  allow_delegation=True,
)

# Create tasks for your agents
# Being explicit on the task to ask for human feedback.

task1 = Task(
  description=f"""Analyze and define ways a theoretical bad actor could exploit the policy framework provided here {texts}. Your final report must be a detail list of all possible exploits and mechanisms. Think step by step.""",
  agent=whitehat,
#tools=tools
)

task2 = Task(
  description="""Implement changes to the insurance terms and conditions on the basis of the exploits found. Your final report should list all changes made and why.""",
  agent=risk_analyst
)


# Instantiate your crew with a sequential process
crew = Crew(
  agents=[whitehat, risk_analyst],
  tasks=[task1, task2],
  verbose=2,
#   process=Process.sequential
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

[DEBUG]: Working Agent: WhiteHat Finance/Risk/Insurance Analyst
[INFO]: Starting Task: Analyze and define ways a theoretical bad actor could exploit the policy framework provided here [Document(page_content='This is not an Insurance Product and approved maintenance requests will have  \nno affect on your comprehensive insurance no-claim bonus.Volkswagen\nCare Plan', metadata={'source': 'policy3.pdf', 'page': 0}), Document(page_content='Volkswagen Care PlanPage 1\nTable of contents\nOverview of our Services 2\n1. Volkswagen Care Plan 2\n2 How to request a maintenance call 2\n3.  Maintenance limit  3\n4.  Maintenance period 3\n5.  Payment of fees 3\n6.  What maintenance is not included? 3\n7.  Cancellation  3 \n8.  Fraud  4\n9. Y our responsibilities 4\n10.  Important terms 4 \n11. Contact details 4\n12. Disclosure Notice 5', metadata={'source': 'policy3.pdf', 'page': 1}), Document(page_content='Page 2\nVolkswagen Care PlanOVERVIEW OF OUR SERVICES\nThe Volkswagen Care Plan is an agreemen